In [4]:
import clickhouse_driver
from clickhouse_driver import Client

from pynput import mouse

import pyautogui
import datetime
from datetime import datetime

import pandas as pd


In [2]:

# connecting to server
connection = clickhouse_driver.connect(
host='localhost',
port=9000,
user='default',
database='mouse_movements')

In [6]:
#creating database
cursor = connection.cursor()
create_database_query = 'CREATE DATABASE IF NOT EXISTS mouse_movements'
cursor.execute(create_database_query)

In [7]:
#creating table for staging
create_table_query = '''
CREATE TABLE movements (
x UInt16,
y UInt16,
delta_x UInt16,
delta_y UInt16,
client_time_stamp Float32,
button Int8,
target String
)ENGINE=MergeTree() ORDER BY client_time_stamp
'''
cursor.execute(create_table_query)

In [8]:
#creating table for buffering
create_table_query = '''
CREATE TABLE movements_buffer AS movements 
ENGINE = Buffer(mouse_movements, movements, 
16, 10, 100, 10000, 1000000, 10000000, 100000000)
'''
cursor.execute(create_table_query)

In [ ]:
#mouse catch-click function
client_df = Client('localhost', settings={'use_numpy':True})

prev_x = 0
prev_y = 0
cols = ['x', 'y', 'delta_x', 'delta_y', 'client_time_stamp', 'buttons', 'target']
clicks = pd.DataFrame(columns=cols)

with mouse.Events() as events:
    for event in events:
        if type(event) == mouse.Events.Click:
            dat = [event.x, event.y, abs(event.x-prev_x), abs(event.y-prev_y), \
                   datetime.now().timestamp(), event.button.value[0],
                   pyautogui.getActiveWindowTitle()]
            clicks.iloc[-1] = dat
            client_df.insert_dataframe(f'INSERT INTO mouse_movements.movements_buffer VALUES', clicks)